In [2]:
import sys
sys.path.insert(0,"../")

In [3]:
from pyspark.sql import SparkSession
from config.config import CONFIG
from etl.extract.extract import extract_raw_data
from etl.transform import parseLogs, convertTimestamp, clean, aggregate, sessionize, categorizeEndpoint
from etl.load.load import load_data

In [4]:
def start_session(appname: str) -> SparkSession:
    return SparkSession.builder.config("spark.driver.memory", "15g").appName(appname).getOrCreate()

In [5]:
def stop_session(spark: SparkSession) -> None:
    spark.stop()

In [6]:
def run_pipeline() -> None:
    spark = start_session(CONFIG['app_name'])
    raw_df = extract_raw_data(spark, CONFIG)
    parsed_df = parseLogs.parse_logs(raw_df, CONFIG)
    categorized_df = categorizeEndpoint.categorize_endpoint(parsed_df, CONFIG)
    converted_df = convertTimestamp.convert_timestamp(categorized_df, CONFIG)
    cleaned_df = clean.clean_data(converted_df)

    sessions_df = sessionize.sessionize_data(cleaned_df, CONFIG)

    traffic_df, error_df = aggregate.calculate_aggregates(cleaned_df)

    load_data(cleaned_df, CONFIG['output']['cleaned_path'], CONFIG)
    load_data(sessions_df, CONFIG['output']['sessionized_path'], CONFIG)
    load_data(error_df, CONFIG['output']['error_analysis_path'], CONFIG)
    load_data(traffic_df, CONFIG['output']['peak_traffic_path'], CONFIG)

    stop_session(spark)

In [7]:
if __name__ == "__main__":
    run_pipeline()

25/03/19 15:39:53 WARN Utils: Your hostname, Harshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.236 instead (on interface en0)
25/03/19 15:39:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/19 15:39:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
